# **Date: 19/07/2021 Day: 5**
# **Title: Intro to REST API**

## **Task:**

### Using public open API website  https://gorest.co.in/, create a python code which can visualize the list users in pandas data frame format and allows option to create new user and delete user. At last at the of the code execution, print a data frame with all actions performed in the python execution cycle.

In [1]:
#Importing all the required modules
from dotenv import load_dotenv
load_dotenv()
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
import os
import json
import sys
from datetime import datetime as dt

In [2]:
#Declaring ENVs and Global Variables
output=os.getenv('OUTPUT')
auth_key=os.getenv('AUTH_KEY')
logcat=[]

In [3]:
#Function for logging the action done by the user
def logs(time,method,function,docs,status,msg,output):
    '''Maintains log info about a particular method'''
    global logcat
    logcat.append(dict(time=time,http_method=method,function=function,description=docs,status_code=status,message=msg,output=output))

In [4]:
#Gets data of all the users in the gorest.co.in website and storing it to csv and json
def get_all_users():
    '''Get the ID, Name, Email, Gender, Status information of all users in the complete database'''
    try:
        print("This method takes long time to finish due to huge data! 🕐")
        all_users=[]
        r=requests.get(f'https://gorest.co.in/public/v1/users')
        print(f"Getting Data from Pages... 🐌")
        for i in range(1,(r.json()["meta"]["pagination"]["pages"])+1):
            temp=requests.get(f'https://gorest.co.in/public/v1/users?page={i}')
            temp=temp.json()
            for j in temp["data"]:
                all_users.append(dict(id=j["id"],name=j["name"],email=j["email"],gender=j["gender"],status=j["status"]))
        print(f"Successfully Retrieved ✅")
        with open(os.path.join(output,'all_users.json'),'w') as aa:
            aa.write(json.dumps(all_users,indent=2))
        df=pd.read_json(os.path.join(output,'all_users.json'))
        df.to_csv(os.path.join(output,'all_users.csv'),index=False)
        op="All User Data Extracted, please find the file at .\output\\all_users.json"
        print(op)
    finally:
        logs(dt.now(),'GET',get_all_users.__name__,get_all_users.__doc__,r.status_code,op,"Its a huge output Find file at .\output\\all_users.csv")

In [5]:
#Get details of a particular user
def get_user(email):
    '''Get User details based on Email'''
    try:
        r=requests.get(f'https://gorest.co.in/public/v1/users?email={email}')
        if r.status_code==200:
            if r.json()["data"]==[]:
                op="No User Found ❌"
                print(op)
            else:
                op="Data Successfully Retrieved ✅"
                print(op)
                print(json.dumps(r.json()["data"],indent=4))
        elif r.status_code>=400:
            op=f"Error Code: {r.status_code}"
            print(op)
            print(json.dumps(r.json()["data"],indent=4))
    finally:
        logs(dt.now(),'GET',get_user.__name__,get_user.__doc__,r.status_code,op,r.json()["data"])


In [6]:
#Create a new user to DB
def create_user(name,email,gender,status):
    '''Creating a new user to DB'''
    payload={
    "access-token": auth_key,
    "name": name,
    "email": email,
    "gender": gender,
    "status": status
    }
    try:
        r=requests.post('https://gorest.co.in/public/v1/users',data=payload)
        if r.status_code==201:
            op=f"🆕 🧑🏽 User {name} Successfully Created ✅"
            print(op)
            print(json.dumps(r.json()["data"],indent=4))
        elif r.status_code>=400:
            op=f"Error Code: {r.status_code}"
            print(op)
            print(json.dumps(r.json()["data"],indent=4))
    finally:
        logs(dt.now(),'POST',create_user.__name__,create_user.__doc__,r.status_code,op,r.json()["data"])

In [7]:
def delete_user(email):
    '''Deleting a new user in the DB'''
    payload={
    "access-token": auth_key
    }
    try:
        r=requests.get(f'https://gorest.co.in/public/v1/users?email={email}')
        if r.status_code==200:
            if r.json()["data"]==[]:
                op="No User Found ❌\nCan't Be Deleted 🗑"
                print(op)
            else:
                r=requests.delete(f'https://gorest.co.in/public/v1/users/{r.json()["data"][0]["id"]}',data=payload)
                if r.status_code==204:
                    op="User Successfully Found ✅ and Deleted 🗑"
                    print(op)
                elif r.status_code>=400:
                    op=f"Error Code: {r.status_code}"
                    print(op)
        elif r.status_code>=400:
            op=f"Error Code: {r.status_code}"
            print(op)
            print(json.dumps(r.json()["data"],indent=4))
    finally:
        logs(dt.now(),'DELETE',delete_user.__name__,delete_user.__doc__,r.status_code,op,op)

In [8]:
def update_user(email):
    ''''Updates name,email,gender,status of already existing data in the DB'''
    try:
        r=requests.get(f'https://gorest.co.in/public/v1/users?email={email}')
        if r.status_code==200:
            if r.json()["data"]==[]:
                op="No User Found ❌\nSo Can't Be Updated 🗑"
                print(op)
            else:
                op="User Successfully Found ✅"
                print(op)
                print(json.dumps(r.json()["data"],indent=4))
                print("\nEnter Updated details: ")
                name=str(input("Please enter Name: "))
                email=str(input("Please enter Email: "))
                gender=str(input("Please enter gender: "))
                status=str(input("Please enter status: "))
                payload={
                    "access-token": auth_key,
                    "name": name,
                    "email": email,
                    "gender": gender,
                    "status": status
                    }
                r=requests.put(f'https://gorest.co.in/public/v1/users/{r.json()["data"][0]["id"]}',data=payload)
                if r.status_code<300:
                    op="User Successfully Updated!! ❇❇"
                    print(op)
                    print(json.dumps(r.json()["data"],indent=4))
                elif r.status_code>=400:
                    op=f"Error Code: {r.status_code}"
                    print(op)
        elif r.status_code>=400:
            op=f"Error Code: {r.status_code}"
            print(op)
            print(json.dumps(r.json()["data"],indent=4))
    finally:
        logs(dt.now(),'UPDATE',update_user.__name__,update_user.__doc__,r.status_code,op,r.json()["data"])

In [9]:
#Quit fn. that quits the program and saves logs
def quit_prg():
    print("Exporting Logs... 🧾🧾")
    global logcat
    log_cat=pd.DataFrame(logcat)
    log_cat.to_csv(os.path.join(output,'logs.csv'),index=False)
    logcat=[]
    print("Logs Exported Successfully! ✅")
    sys.exit(0)
    

In [10]:
#Driver Code
def engine_start():
    if not os.path.exists(output):
        os.makedirs(output)
    while(1):
        print('''\t1. Get all users in the database
            2. Get details about particular user
            3. Create a new User
            4. Delete existing user
            5. Update existing user details
            6. Quit''')
        choices=[1,2,3,4,5,6]
        choice=int(input('\nPlease enter your choice: '))
        if choice==1:
            get_all_users()
        if choice==2:
            get_user(str(input("Please enter Email address of the user to be found: ")))
        if choice==3:
            name=str(input("Please enter Name: "))
            email=str(input("Please enter Email: "))
            gender=str(input("Please enter gender: "))
            status=str(input("Please enter status: "))
            create_user(name,email,gender,status)
        if choice==4:
            delete_user(str(input("Please enter Email address of the user to be deleted: ")))
        if choice==5:
            update_user(str(input("Please enter Email address of the user to be updated: ")))
        if choice==6:
            quit_prg()
        if choice not in choices:
            print('Please enter a valid choice!!')
            engine_start()

In [11]:
engine_start()

	1. Get all users in the database
            2. Get details about particular user
            3. Create a new User
            4. Delete existing user
            5. Update existing user details
            6. Quit
This method takes long time to finish due to huge data! 🕐
Getting Data from Pages... 🐌
Successfully Retrieved ✅
All User Data Extracted, please find the file at .\output\all_users.json
	1. Get all users in the database
            2. Get details about particular user
            3. Create a new User
            4. Delete existing user
            5. Update existing user details
            6. Quit
Error Code: 422
[
    {
        "field": "email",
        "message": "has already been taken"
    }
]
	1. Get all users in the database
            2. Get details about particular user
            3. Create a new User
            4. Delete existing user
            5. Update existing user details
            6. Quit
🆕 🧑🏽 User Post Malone Successfully Created ✅
{
    "id": 1530,
  

SystemExit: 0

C:\Users\vijay\anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
